# 5章 誤差逆伝播法

前章では、ニューラルネットワークの重みパラメータの勾配（重みパラメータに関する損失関数の勾配）は、数値微分によって求めました。数値微分は実装が簡単ですが、計算時間がかかるという難点があります。本章では、重みパラメータの勾配の計算を効率的に行う<b>「誤差逆伝播法」</b>について学びます。<br>誤差逆伝播を正しく理解するためには、「数式」のよる方法と<b>「計算グラフ」</b>による方法がありますが、本章では計算グラフによって<b>"視覚的"</b>に誤差逆伝播法を学びます。
<br>なお、誤差逆伝播法を計算グラフによって説明するアイディアは、Andrej Karpathyのブログ、Fei-Fei Li教授らによって行われたスタンフォード大学のディープラーニングの授業「CS231n」を参考にしています。

# 5.1 計算グラフ

計算グラフとは、計算の過程をグラフによって表したものです。複数のノード（○）とエッジ（ノード間を結ぶ直線）で表現されます。本節では、計算グラフに慣れるため簡単な問題を解きます。

## 5.1.1 計算グラフで解く

問１：太郎くんはスーパーで１個100円のリンゴを2個買いました。支払う金額を求めなさい。ただし、消費税10%が適用されるものとします。
<br><br>
これを計算グラフで解くと次のようになります。
<br><br>
![5-1計算グラフによる問1の答え](".\img\5-1計算グラフによる問1の答え.png")
<img src = ".\img\5-1計算グラフによる問1の答え.png" width="700" height="800">
<br><br>
ノードは丸の中に演算の内容を書き、エッジの上部に計算の途中結果を書くことで、ノードごとの計算結果が左から右へ伝わるように表します。問１では、最初にリンゴの100円が「×2」ノードへ流れ、200円になって次のノードに伝達されます。続いて、その200円が「×1.1」ノードへ流れ、答えの220円が求まります。
<br>なお、上記の例ではノードの中に「×2」や「×1.1」を記載し、ひとつの演算としましたが、乗算である「×」だけを演算としてノードで表すこともできます。その場合、問１は次のようになります。
<br><br>
<img src = ".\img\5-2計算グラフによる問1の答え2.png" width="700" height="800">
<br><br>

問2：太郎くんはスーパーでリンゴを2個、ミカンを3個買いました。リンゴは1個100円、ミカンは1個150円です。消費税が10％かかるものとして、支払う金額を求めなさい。
<br><br>
<img src = ".\img\5-3計算グラフによる問2の答え.png" width="700" height="800">
<br><br>
この問題では、加算ノードである「＋」が新しく加わり、リンゴとミカンの金額を合算します。計算グラフを構成したら、左から右へと計算を進めていきます。リンゴの100円「×2」がノードへ流れ200円になり、ミカンの150円「×3」がノードへ流れ450円となり、次の加算ノードで加算され650円となります。最後に1.1を乗算するノードに流れ答えの715円が求まります。
<br>
このように計算グラフを使って問題を解くには、<br>
１．計算グラフを構築する<br>
２．計算グラフ上で計算を左から右へ進める<br>
という流れで行います。<br>
ここで、2．の「計算を左から右へ進める」というステップを<b>順伝播(forward propagation)</b>と呼びます。反対に、右から左方向への伝播を<b>逆伝播（backward propagation)</b>といいます。

## 5.1.2 局所的な計算

計算グラフの特徴は、「局所的な計算」を伝播することで最終的な結果を得られることです。局所的とは、「自分に関係する小さな範囲」ということをを意味します。具体的に以下のスーパーでリンゴ2個と、それ以外にたくさんの買い物をした場合を例に説明します。たくさんの買い物を行ってその金額が4,000円であったとします。これにリンゴ2個を購入した200円を加算することで、4,200円という合計値が求まります。
<br>ここで大切なポイントは、各ノードにおける計算は局所的な計算であるということです。加算のノードではリンゴ以外の買い物がどのようにされてきたのか（たとえ非常に複雑な計算があったとしても）を、全く考える必要がなく、入力である4,000円と200円という2つの数字の足し算だけを行えばよいのです。
<br><br>
<img src = ".\img\5-4リンゴとそれ以外たくさんの買い物の例.png" width="700" height="800">
<br><br>
このように、計算グラフでは、全体の計算がどんなに複雑であっても、各ステップでやることは、対象とするノードの<br>「局所的な計算」</br>です。その結果を伝達することで、全体を構成する複雑な計算の結果が得られます。

## 5.1.3 なぜ計算グラフで解くのか？

計算グラフの利点は3つあります。<br>
１．局所的な計算・・・全体がどんなに複雑でも各ノードでは局所的に単純な計算を行うことができ、問題を単純化できます。<br>
２．途中の計算結果を保持・・・例えば、リンゴ2個購入まで計算を進めた時の金額は200円などといった具合に。<br>
３．逆伝播により「微分」を効率良く計算できる<br>
この３つの利点の中では、3番が計算グラフを使う最大の理由です。それでは、問１をもう一度使って逆伝播による「微分」を考えてみましょう。
<br><br>
<img src = ".\img\5-5逆伝播による微分値の伝達.png" width="700" height="800">
<br><br>
例えば、リンゴの値段が値上がりした場合、最終的な支払金額にどのように影響するか知りたいとしましょう。これは、<b>「リンゴの値段に関する支払い金額の微分」</b>を求めることに相当します。記号で表記すれば、リンゴの値段をx、支払い金額をLとした場合、$\frac{\partial L}{\partial x}$を求めることに相当します。
<br>上記図では、逆伝播により<b>「局所的な微分」</b>の値が右から左へ<b>「1→1.1→2.2」</b>と伝播されています（上記図の緑色線）。この結果から、<b>「リンゴの値段に関する支払い金額の微分」</b>の値は2.2であることがわかります。これは、リンゴが1円値上がりしたら、最終的な支払金額が2.2円増えることを意味しています。このほかにも、①「消費税に関する支払金額の微分」や②「リンゴ個数に関する支払金額の微分」も同様の手順で求めることができます。（①「1→200」、②「1→1.1→110」）
<br>このように、計算グラフを用いれば、順伝播と逆伝播により、各変数の微分の値を効率よく求めることができる点にあります。

# 5.2 連鎖律

前節で述べた、左から右への逆伝播で「局所的な微分」を伝達する原理は、<b>連鎖律(chain rule)</b>によるものです。本節では連鎖律について説明し、それが計算グラフ上での逆伝播に対応することを明らかにします。

## 5.2.1 計算グラフの逆伝播

ここでは,$y = f(x)$の逆伝播を考えます。
<br><br>
<img src = ".\img\5-6計算グラフの逆伝播.png" width="500" height="800">
<br><br>
上図に示す通り、逆伝播の計算手順は、信号Ｅに対して、ノードの局所的な微分($\frac{ \partial y }{ \partial x }$)を乗算し、それを前のノードへ伝達していきます。例えば、$y=f(x)=x^2$だったとしたら、$\frac{ \partial y }{ \partial x }$ = 2xになります。そして、その局所的な微分を上流から伝達された値（この例ではＥ）に乗算して、前のノードへと渡していくのです。
<br>この計算をすることで、なぜ目的とする微分の値を効率良く求めることができるのかは、連鎖律の原理から説明できます。

## 5.2.2 連鎖律とは

連鎖律を説明するためには、まず<b>合成関数</b>を理解する必要があります。合成関数とは、複数の関数によって構成される関数のことです。例えば、$z = (x + y)^2$という式は次のように2つの式で構成されていると考えることができます。
<br>
$$z = t^2$$
$$t = x + y$$
<br>
連鎖律とは合成関数の微分についての性質であり、次のように定義されます。
<br>
<br>
<b>ある関数が合成関数で表される場合、その合成関数の微分は、合成関数を構成するそれぞれの関数の微分の積によって表すことができる。</b>
<br>
<br>
上記の例でいうと、$\frac{\partial z}{\partial x}$（xに関するzの微分）は、$\frac{\partial z}{\partial t}$（tに関するzの微分）と$\frac{\partial t}{\partial x}$（xに関するtの微分）の積によって表すことができる。ということです。これを数式で表すと、
$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$$
と書くことができます。それでは、連鎖律を使って、$z = (x + y)^2$の微分$\frac{\partial z}{\partial x}$を求めてみましょう。それには、
<br>
$$z = t^2$$
$$t = x + y$$
<br>
の局所的な微分（偏導関数）を先に求めます。
<br>
$$\frac{\partial z}{\partial t} = 2t$$
$$\frac{\partial t}{\partial x} = 1$$
<br>
そして、最終的に求めたい$\frac{\partial z}{\partial x} $は、上記で求めた偏導関数の積によって計算できます。
$$\frac{\partial z}{\partial t} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t ・ 1 = 2(x + y)$$

# 5.2.3 連鎖律と計算グラフ

それでは、前節で行った連鎖律の計算を、計算グラフで表してみましょう。
<br><br>
<img src = ".\img\5-7計算グラフの逆伝播_局所的微分の乗算と伝播.png" width="500" height="800">
<br><br>
逆伝播は右から左へと信号を伝播していきます。逆伝播の計算手順では、ノードへの入力信号に対してノードの局所的な微分（偏微分）を乗算して次のノードへと伝達していきます。例えば、「＊＊２」への逆伝播時の入力は$\frac{\partial z}{\partial z}$であり、これに局所的な微分（偏微分）である$\frac{\partial z}{\partial t}$を乗算して、次のノード「＋」へ渡します。
<br>ここで、一番左の逆伝播の結果に注目してみましょう。$\frac{\partial z}{\partial z} \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = \frac{\partial z}{\partial x}$が成り立ち「ｘに関するｚの微分」に対応しています。つまり、逆伝播が行っていることは、連鎖律の原理から構成されているのです。
<br>なお、上図に前節で求めた偏導関数の結果を代入すると次のようになります。
<br><br>
<img src = ".\img\5-8計算グラフの逆伝播の結果.png" width="500" height="800">
<br>

# 5.3 逆伝播

前節では、連鎖律によって逆伝播が成り立つことを説明しました。本節では、「＋」や「×」などの演算を例に、逆伝播の仕組みについて説明します。

## 5.3.1 加算ノードの逆伝播

初めに加算ノードの逆伝播について、z = x + yという式を例に考えます。この式の微分は、
$$\frac{\partial z}{\partial x} = 1 $$
$$\frac{\partial z}{\partial y} = 1  $$
です。この結果から、計算グラフを次のように作成することができます。
<br><br>
<img src = ".\img\5-9加算ノードの逆伝播.png" width="500" height="800">
<br><br>
逆伝播の際には、上流から伝わった微分（$\frac{\partial L}{\partial z}$）に「１」を乗算して下流に流します。つまり加算ノードの逆伝播は「１」を乗算するだけなので、入力値をそのまま次のノードに流すだけになります。
<br>この例では、上流から伝わった微分の値を$\frac{\partial L}{\partial z}$としましたが、これは、次の図に示すように最終的にＬという値を出力する大きな計算グラフを想定しているためです。
<br><br>
<img src = ".\img\5-10加算ノードの順伝播.png" width="500" height="800">
<br><br>

それでは、加算の逆伝播の具体例として「１０ + ５ = １５」を見てみましょう。
<br><br>
<img src = ".\img\5-11加算ノードの逆伝播の具体例.png" width="500" height="800">
<br><br>
例えば、上流から「1.3」という値が流れ来たとしましょう。（ここでは1.3という値に意味はない？）加算ノードの逆伝播は次のノードへそのまま出力するだけなので、「1.3」を次のノードへ渡します。

## 5.3.2 乗算ノードの逆伝播

乗算ノードの逆伝播について、z = xy という式を例に考えます。この式の微分は、
$$\frac{\partial z}{\partial x} = y $$
$$\frac{\partial z}{\partial y} = x  $$
です。この結果から、計算グラフを次のように作成することができます。
<br><br>
<img src = ".\img\5-12乗算ノードの逆伝播.png" width="500" height="800">
<br><br>
乗算の逆伝播の場合は、上流の値に順伝播の際の入力信号を「ひっくり返した値」を乗算して下流へ流します。それでは具体例「10×5=50」を見てみましょう。
<br><br>
<img src = ".\img\5-13乗算ノードの逆伝播の具体例.png" width="500" height="800">
<br><br>
加算ノードの場合と同様に上流から「1.3」という値が流れて来たとしましょう。（乗算と同様に1.3という値に意味はない？）乗算の逆伝播は、入力信号をひっくり返した値を乗算するので、「1.3 × 5=  6.5」、「1.3 × 10 = 13」となります。このように乗算ノードの逆伝播では順伝播の入力信号が必要となるので、実装時にも順伝播の入力信号を保持します。

## 5.3.3 リンゴの例

ここで改めて、冒頭に見たリンゴの買い物の例を考えます。ここで解きたい問題は、リンゴの値段、リンゴの個数、消費税の３つの変数それぞれが最終的な支払金額にどのように影響するか、ということです。これは、「リンゴの値段に関する支払い金額の微分」、「リンゴの個数に関する支払い金額の微分」、「消費税に関する支払い金額の微分」を求めることに相当します。これを逆伝播で表すと次のようになります。
<br><br>
<img src = ".\img\5-14リンゴの買い物の逆伝播の例.png" width="500" height="800">
<br><br>
図からわかるように、「リンゴの値段に関する支払い金額の微分」は２.２、「リンゴの個数に関する支払い金額の微分」は１１０、「消費税に関する支払い金額の微分」２００となります。これは、例えば、リンゴの値段が1円増えれば支払い金額が2.2円増加することを示し、リンゴの個数が1個増えれば支払い金額が１１０円増えること示します。
<br>それでは、最後に練習問題として、以下の図「リンゴとミカンと逆伝播の例」を解いてみましょう。
<br><br>
<img src = ".\img\5-15リンゴとミカンの買い物の逆伝播の例.png" width="500" height="800">
<br><br>
<br>===================================この下は回答なので見ないでね！=================================================
<br><br>
<br><br>
<br><br>
<br><br>
<br><br>
<br><br>
<br><br>
回答は次の通りです。試しに、ミカンの購入個数を2個増やしてみましょう。「ミカンの個数に関する支払い金額の微分」は１６５円なので、２個増えた場合、支払い金額が３３０円増えるはずです。これを順伝播で確認してみましょう。右下の青枠にある通り、リンゴ2個、ミカン5個（2個増加）の支払い金額は1,045円となり、元々の金額（715円）より330円増えていることが確認できます。
<img src = ".\img\5-15リンゴとミカンの買い物の逆伝播の例_回答.png" width="500" height="800">
<br><br>


# 5.4 単純なレイヤの実装

本節では、これまで見てきた「リンゴの買い物」の例を、Pythonで実装します。ここでは計算グラフの乗算ノードを「乗算レイヤ(MulLayer)」、加算ノードを「加算レイヤ(AddLayer)」という名前で実装することにします。
<br><br>
※次節では、ニューラルネットを構成する「層（レイヤ）」を一つのクラスで実装します。ここで言う「レイヤ」とは、ニューラルネットワークにおける機能の単位です。例えば、シグモイド関数のための<b>Sigmoidレイヤ</b>や、行列の内積のための<b>Affineレイヤ</b>などです。

## 5.4.1 乗算レイヤの実装

レイヤは、forward()：順伝播とbackward()：逆伝播という共通のメソッド（インターフェース）を持つように実装します。

In [2]:
#乗算レイヤをクラスとして実装
class MulLayer:
    #インスタンス生成（ｘとｙの初期化。また順伝播時の入力値を保持するために使用する）
    def __init__(self):
        self.x = None
        self.y = None
        
    #順伝播メソッド(XとYを乗算して返すだけ)
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    ##逆伝播メソッド(上流から引き継がれた微分doutに対してｘとｙを逆にして乗算した値を返す)
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

それでは、この乗算レイヤを使ってこれまで見てきたリンゴの買い物を実装してみましょう。
<br><br>
<img src = ".\img\5-14リンゴの買い物の逆伝播の例.png" width="500" height="800">
<br><br>

In [6]:
apple = 100.0 #リンゴの値段
apple_num = 2.0 #リンゴの個数
tax = 1.1 #消費税

#layer
mul_apple_layer = MulLayer() #リンゴレイヤ（最初の乗算）のインスタンスを生成
mul_tax_layer = MulLayer() #消費税レイヤ（最後の乗算）のインスタンスを生成

#forward:順伝
apple_price = mul_apple_layer.forward(apple, apple_num) #リンゴの値段×リンゴの個数を実行
price = mul_tax_layer.forward(apple_price, tax) #上記の計算結果×消費税（1.1）を実行

print(round(price)) #支払い額を表示(変な誤差がでるのでRoundで丸める)

220


また、各変数に関する微分（逆伝播）はbackward()で求めることができます。

In [15]:
#backward：逆伝播
dprice = 1 #支払い額の逆伝播値（逆伝播の初期値）
#消費税乗算前の合計値に関する支払い額の微分と、消費税に関する支払い額の微分を求める
dapple_price, dtax = mul_tax_layer.backward(dprice)
#リンゴの値段に関する支払い額の微分と、リンゴの個数に関する支払い額の微分を求める（入力は上記の微分結果：dapple）
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print('消費税に関する支払い額の微分：' + str(round(dtax,1)))
print('消費税乗算前の合計値に関する支払い額の微分：' + str(round(dapple_price,1)))
print('リンゴの値段に関する支払い額の微分：' + str(round(dapple,1)))
print('リンゴの個数に関する支払い額の微分：' + str(round(dapple_num,1)))


消費税に関する支払い額の微分：200.0
消費税乗算前の合計値に関する支払い額の微分：1.1
リンゴの値段に関する支払い額の微分：2.2
リンゴの個数に関する支払い額の微分：110.0


## 5.4.2 加算レイヤの実装

続いて足し算ノードである加算レイヤを実装します。

In [16]:
#加算レイヤをクラスとして実装
class AddLayer:
    #インスタンス生成（pass:特に何もしないということ）
    def __init__(self):
        pass
        
    #順伝播メソッド(XとYを加算して返すだけ)
    def forward(self, x, y):
        out = x + y
        
        return out
    
    ##逆伝播メソッド(上流から引き継がれた微分doutに対して1を乗算した値を返す：そのまま返す)
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

それでは、加算レイヤと乗算レイヤを使って、リンゴ2個とミカン3個の買い物を実装しましょう。
<br><br>
<img src = ".\img\5-17リンゴ2個とミカン3個の買い物.png" width="500" height="800">
<br><br>

In [24]:
apple = 100 #リンゴ値段
apple_num = 2 #リンゴ個数
orange = 150 #ミカン値段
orange_num = 3 #ミカン個数
tax = 1.1 #消費税

#layer 各層の定義（インスタンス化）
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward：順伝播
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print('順伝播の結果：' + str(round(price,1)))

#backward:逆伝播
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(
    '最終合計金額に関する支払い額の微分：' + str(round(dall_price,1)) + '\r\n', 
    '消費税に関する支払い額の微分：' + str(round(dtax,1)) + '\r\n',
    'リンゴ合計金額に関する支払い額の微分：' +  str(round(dapple_price,1)) + '\r\n', 
    'ミカン合計金額に関する支払い額の微分：' + str(round(dorange_price,1)) + '\r\n',
    'リンゴの値段に関する支払い額の微分：' + str(round(dapple,1)) + '\r\n', 
    'リンゴの個数に関する支払い額の微分：' + str(round(dapple_num,1)) + '\r\n',
    'ミカンの値段に関する支払い額の微分：' + str(round(dorange,1)) + '\r\n', 
    'ミカンの個数に関する支払い額の微分：' + str(round(dorange_num,1))
)

順伝播の結果：715.0
最終合計金額に関する支払い額の微分：1.1
 消費税に関する支払い額の微分：650
 リンゴ合計金額に関する支払い額の微分：1.1
 ミカン合計金額に関する支払い額の微分：1.1
 リンゴの値段に関する支払い額の微分：2.2
 リンゴの個数に関する支払い額の微分：110.0
 ミカンの値段に関する支払い額の微分：3.3
 ミカンの個数に関する支払い額の微分：165.0


このように、、計算グラフにおけるレイヤの実装は簡単に行うことができ、それらを使えば複雑な微分の計算を求めることができます。次節ではニューラルネットで使われるレイヤを実装します。